# BASELINE MODEL

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import os
import numpy as np
import scipy
import sys
from scipy import io
import cv2
import math
import random
import keras
from sklearn import preprocessing
from glob import glob

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# GLOBALS

In [2]:
POET_DIR = "D:/data/POETdataset/"
PATCHES_FILE = os.path.join(POET_DIR, "x.npy")
LABELS_FILE = os.path.join(POET_DIR, 'y.npy')
pascal_images = os.path.join(POET_DIR, 'PascalImages')
PATCH_SIZE = 32
HALF_PATCH = PATCH_SIZE//2
DATASET_SIZE = 6270
VALIDATION_PERC = 0.2
MAX_GAZE_POINTS = 3 # TEMPORARY
MIN_GAZE_POINTS = 3
VGG_NR_FEATURES = 512
user_index = 0 # TEMPORARY
BATCH_SIZE = 50
IMG_SIZE = 224

In [3]:
os.chdir(POET_DIR)

# DATA 

In [4]:
classes = [a.split("_")[1].split(".mat")[0] for a in glob("etData/*")]
idx2class = {i:c for i, c in enumerate(classes)}
class2idx = {c:i for i, c in enumerate(classes)}

AttributeError: 'function' object has no attribute 'glob'

In [ ]:
def load_dataset():
  x = []
  y = []
  filenames = glob("PascalImages/*")
  for fn in filenames:
    for cname in classes:
      if cname in fn:
        y_fn = class2idx[cname]
        y.append(y_fn)
        x.append(fn)
        continue
  assert len(y) == len(x)
  return np.array(x),np.array(y) 

In [ ]:
x, y = load_dataset()

In [ ]:
plt.hist(y)

In [ ]:
preds = np.repeat(-1, len(y))

In [ ]:
X_img = np.zeros((len(x), IMG_SIZE, IMG_SIZE, 3))

In [ ]:
for i, fn in enumerate(x):
  img = keras.preprocessing.image.load_img(fn, target_size=(IMG_SIZE, IMG_SIZE, 3))
  img = keras.preprocessing.image.img_to_array(img)
  X_img[i] = img

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
enc = OneHotEncoder()
y = y.reshape(-1,1)
enc.fit(y)
y_onehot = enc.transform(y)

# MODEL

In [ ]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

In [ ]:
model = keras.Sequential()
model.add(vgg16)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy',
                optimizer='adadelta',
                metrics=['accuracy'])

In [ ]:
model.fit(
         X_img,
         y_onehot,
         validation_split=VALIDATION_PERC,
         epochs=10
)